In [6]:
import pandas as pd
import numpy as np
import datetime as dt
import json
from datetime import datetime

import plotly.io as pio
import plotly.graph_objects as go
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')


# this enables relative path imports
import os
from dotenv import load_dotenv
load_dotenv()
_PROJECT_PATH: str = os.environ["_project_path"]
_PICKLED_DATA_FILENAME: str = os.environ["_pickled_data_filename"]

import sys
from pathlib import Path
project_path = Path(_PROJECT_PATH)
sys.path.append(str(project_path))

import config_v2 as cfg

from library_report_v2 import Cleaning as cln
from library_report_v2 import Graphing as grp
from library_report_v2 import Processing as pro
from library_report_v2 import Configuration as repcfg

In [7]:
def show_response_contents(df):
    print("The response contains:")
    print(json.dumps(list(df['variable'].unique()), sort_keys=True, indent=4))
    print(json.dumps(list(df['device'].unique()), sort_keys=True, indent=4))

In [8]:
df = pd.read_pickle(project_path / 'data' / _PICKLED_DATA_FILENAME)
#show_response_contents(df)

In [9]:
df = df.sort_values(by=['variable','datetime'])
df = pro.datetime_attributes(df)

#df_bl, df_st = pro.split_into_baseline_and_study(df, baseline=cfg.BASELINE, study=cfg.STUDY, inclusive='left')
#df_bl['Periodo'] = 'Baseline'
#df_st['Periodo'] = 'Estudio'

#past_w = df_bl.loc[cfg.PAST_WEEK[0]:cfg.PAST_WEEK[1]]

In [13]:
#Del dataframe completo (baselin-study) se saca solo la variable energía activa
front = df.query("variable == 'front-consumo-activa'").copy()

#Se agrupan los datos por intervalos de una hora
front_hour = front.groupby(by=["variable", "device_name"]).resample('1h').sum().round(2).reset_index().set_index('datetime')
front_hour = pro.datetime_attributes(front_hour)


In [17]:
front_hour

,variable,device_name,value,hour,day,month,year,dow
datetime,,,,,,,,
2023-01-01 00:00:00-05:00,front-consumo-activa,BC 044 - Piedecuesta,2.75,0,1,1,2023,domingo
2023-01-01 01:00:00-05:00,front-consumo-activa,BC 044 - Piedecuesta,2.71,1,1,1,2023,domingo
2023-01-01 02:00:00-05:00,front-consumo-activa,BC 044 - Piedecuesta,2.72,2,1,1,2023,domingo
2023-01-01 03:00:00-05:00,front-consumo-activa,BC 044 - Piedecuesta,2.69,3,1,1,2023,domingo
2023-01-01 04:00:00-05:00,front-consumo-activa,BC 044 - Piedecuesta,2.67,4,1,1,2023,domingo
...,...,...,...,...,...,...,...,...
2024-05-19 20:00:00-05:00,front-consumo-activa,BC 90 - Megamall,1.82,20,19,5,2024,domingo
2024-05-19 21:00:00-05:00,front-consumo-activa,BC 90 - Megamall,1.85,21,19,5,2024,domingo
2024-05-19 22:00:00-05:00,front-consumo-activa,BC 90 - Megamall,1.76,22,19,5,2024,domingo


In [16]:
# Seleccionar columnas específicas directamente
front_hour = front_hour[['variable', 'device_name', 'value', 'hour', 'day', 'month', 'year', 'dow']]


In [18]:
front_hour.to_clipboard(index=True, header=True)

In [ ]:
#front_hour = front.groupby(['device_name', 'hour']).agg({'value': 'mean', 'day': 'first', 'dow':'first',
#       'cont_dow':'first', 'week':'first', 'year':'first' }).reset_index().round(2)